## 3.1 텐서와 그래프 실행

In [2]:
### Import Tensorflow as name 'tf'
import tensorflow as tf

In [3]:
### Constant Variable
hello = tf.constant('Hello, Tensorflow!')

print(hello)

Tensor("Const:0", shape=(), dtype=string)


#### 변수 hello가 Tensor라는 변수 type을 가지며, Constant를 담고 있다. Tensor는 가장 기본적이고 중요한 자료형이다.

### Rank, Shape 개념
* Rank는 Matrix의 대괄호를 몇 개까지 뚫을 수 있느냐.
* Shape는 Matrix의 대괄호를 하나씩 뚫을 때마다, 원소의 개수가 몇개 있느냐.

#### ex) [[[1, 2, 4], [2, 3, 5]], [[1, 7, 4], [2, 3, 6]]]는 Rank가 3, Shape는 [2, 2, 3]

#### dtype은 해당 텐서에 담긴 요소들의 자료형. ex) string, float, int 등

In [4]:
a = tf.constant(10)
b = tf.constant(20)

### Add two tensors
c = tf.add(a, b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


#### 결과를 보면, 상수 텐서 두 개가 더해지지 않았는데, 이는 Tensorflow가 그래프 생성과 실행이 코드에서 분리되어 있기 때문이다.
#### 그래프는 텐서들의 연산 모음.
#### 텐서와 텐서들의 연산들을 코드에서 미리 정의하여 그래프를 만듬. &rarr; 연산을 실행하는 코드를 넣어 원하는 시점에 실제 연산을 수행.
#### 이러한 연산 방식을 지연 실행(Lazy Evaluation)이라 한다.

#### 그래프의 연산 실행은 Session 안에서 이뤄져야 하며, Session 객체와 run 메서드를 사용한다.

In [5]:
### Make Session instance
sess = tf.Session()

### Run Session
print(sess.run(hello))
print(sess.run([a, b, c]))

### Close the Session
sess.close()

b'Hello, Tensorflow!'
[10, 20, 30]


## 3.2 Placeholder와 Variable

* Placeholder: 그래프에 사용할 **입력값을 나중에 받기 위해 사용**하는 매개변수.
* Variable: 그래프를 최적화하는 용도로, Tensorflow의 학습 함수들이 **학습 결과를 갱신하기 위해 사용**하는 변수

In [6]:
### None means that size is undetermined
X = tf.placeholder(tf.float32, [None, 3])
print(X)

### Future input data
x_data = [[1, 2, 3], [4, 5, 6]]

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


#### X는 Placeholder로 나중에 training set을 넣을 자료로 정의 

In [7]:
### Define variable
W = tf.Variable(tf.random_normal([3, 2]))
### You can define W implicitly like
### W = tf.Variable([[0.1, 0.1], [0.1, 0.2], [0.2, 0.3]])
b = tf.Variable(tf.random_normal([2, 1]))

### Define operation expression
### Matirx multiplication: X * W + b
expr = tf.matmul(X, W) + b

#### random_normal은 정규분포의 임의의 값으로 초기화한다.

In [8]:
sess = tf.Session()
### Initialize variables
sess.run(tf.global_variables_initializer())

print("--- x_data ---")
print(x_data)
print("--- W ---")
print(sess.run(W))
print("--- b ---")
print(sess.run(b))
print("--- expr ---")
### Link placeholder X with pre-defined x_data
print(sess.run(expr, feed_dict = {X: x_data}))

sess.close()

--- x_data ---
[[1, 2, 3], [4, 5, 6]]
--- W ---
[[ 1.6845267  -0.7097309 ]
 [ 1.7618015   0.59761894]
 [ 1.472614    0.07079584]]
--- b ---
[[-0.5572125]
 [ 0.5276721]]
--- expr ---
[[ 9.068759    0.14068198]
 [24.91047     1.1016183 ]]


#### global_variables_initializer는 앞에서 정의한 변수들을 초기화하는 함수이다. 변수를 처음 실행하는 것이라면, 연산을 실행하기 전에 반드시 이 함수를 호출한다.

#### feed_dict 매개변수는 그래프를 실행할 때 사용할 입력값을 지정한다. Placeholder X에 미리 정의해둔 x_data를 넣어준다.

## 3.3 선형 회귀 모델 구현하기

#### 선형 회귀: 주어진 x와 y 값을 가지고 서로 간의 관계를 파악한 뒤에, 새로운 x 값에 대한 y 값을 예측하는 것.

In [9]:
### Training Sets
x_data = [1, 2, 3]
y_data = [1, 2, 3]

In [10]:
### Initialize weight & bias randomly in the uniform distribution
W = tf.Variable(tf.random_uniform([1], -1, 1))
b = tf.Variable(tf.random_uniform([1], -1, 1))

In [11]:
### Placeholders
X = tf.placeholder(tf.float32, name = "X")
Y = tf.placeholder(tf.float32, name = "Y")

print(X)
print(Y)

Tensor("X:0", dtype=float32)
Tensor("Y:0", dtype=float32)


#### Placeholder의 name 매개변수로 플레이스홀더의 이름을 설정할 수 있다. name이 없어도 자동으로 이름은 부여된다.

In [12]:
### Linear hypothesis function
hypothesis = W * X + b

#### W와 X가 아직 행렬이 아니므로, matmul 메소드가 아닌 기본 곱셈 연산자를 사용했다.

In [13]:
### Cost Function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

#### 가설 함수값과 실제값의 차이를 제곱하여 평균한 것을 Cost Function으로 삼겠다.

In [14]:
### Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)
train_op = optimizer.minimize(cost)

#### 경사하강법은 함수의 기울기를 구하고 기울기가 낮은 쪽으로 계속 이동시키면서 최적의 값을 찾아 나가는 방법이다.
#### 학습률은 얼마나 빠르게 이동시킬 것인가를 설정하는 값이다. &rarr; 학습을 진행하는 과정에 영향을 주는 변수를 hyperparameter(하이퍼파라미터)라 한다. 이것을 잘 튜닝하는 것 역시 큰 과제.

In [15]:
### session block with "with" block
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        ## _ is assigned to train_op & cost_val is assigned to cost
        _, cost_val = sess.run([train_op, cost], feed_dict = {X: x_data, Y: y_data})
        
        if step % 20 == 0:
            print(step, cost_val, sess.run(W), sess.run(b))

0 9.909205 [1.309602] [-0.34144145]
20 0.009113527 [1.108211] [-0.2459892]
40 0.0034433426 [1.0665147] [-0.15120374]
60 0.0013009881 [1.040885] [-0.09294134]
80 0.0004915465 [1.0251311] [-0.05712883]
100 0.0001857205 [1.0154475] [-0.03511575]
120 7.0170136e-05 [1.0094951] [-0.02158478]
140 2.6512205e-05 [1.0058365] [-0.01326773]
160 1.0017253e-05 [1.0035875] [-0.00815533]
180 3.7847187e-06 [1.0022051] [-0.00501287]
200 1.4299598e-06 [1.0013554] [-0.00308127]
220 5.402896e-07 [1.0008332] [-0.00189398]
240 2.0411345e-07 [1.0005121] [-0.00116417]
260 7.715153e-08 [1.0003148] [-0.00071562]
280 2.9128396e-08 [1.0001935] [-0.00043983]
300 1.0999771e-08 [1.000119] [-0.00027037]
320 4.1567696e-09 [1.0000731] [-0.00016613]
340 1.5655909e-09 [1.000045] [-0.00010204]
360 5.954585e-10 [1.0000275] [-6.2703664e-05]
380 2.2342898e-10 [1.0000169] [-3.8547885e-05]
400 8.492407e-11 [1.0000105] [-2.374209e-05]
420 3.193179e-11 [1.0000064] [-1.4618601e-05]
440 1.2165676e-11 [1.0000039] [-9.023713e-06]
460